## Closure
Anna Mackie, 2022

A script to check that the '3D' method of wa500 selection and dynamic effect. Looping through each model, the data is decomposed into dynamic, thermodyanmic and nonlinear components. These are then recombined and checked against total cloud feedback (calculated directly) to ensure the differences are negligable.

## Note on method of R(w) and A(w) calculation
Please see Methods section for more detail. Approach is to 
1. load 3D (level, x, y) pressure and vertical velocity fields
2. find, at each grid point (x, y), the level which is closest to 500 hPa in the pressure field
3. use this (level, x, y) to extract vertical velocity at ~ 500 hPa
4. sort vertical velocity at 500 hPa into vertical velocity 'bins' using np.digitize
5. take mean of longwave and shortwave CRE of gridpoints within each vertical velocity 'bin' to create R(w) functions
6. take proportion of gridpoints within each vertical velocity 'bin' to create A(w) function.## Note on method of R(w) and A(w) calculation


No figures produced.

In [1]:
# activate virtual environmnet required for metpy
import sys
import pathlib
import platform
venv_path = '~/nb-venvs/metpy_venv'
sys.path.append(str(pathlib.Path(f'{venv_path}/lib/python{platform.python_version_tuple()[0]}.{platform.python_version_tuple()[1]}/site-packages/').expanduser()))

import metpy
print(metpy.__file__)

/home/users/arm33/nb-venvs/metpy_venv/lib/python3.10/site-packages/metpy/__init__.py


In [2]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from metpy.calc import saturation_mixing_ratio
from metpy.units import units
from scipy import stats
import matplotlib.lines as mlines
import matplotlib as mpl
mpl.rcParams['font.size'] = 16

import sys
sys.path.append('../')
import funcs



In [3]:
models = ['CM1', 'dam','ICON_LEM_CRM','ICON_NWP_CRM','MESONH' ,'SAM_CRM', 'SCALE', 
           'UCLA-CRM','UKMOi-vn11.0-CASIM' , 'UKMOi-vn11.0-RA1-T', 
          'UKMOi-vn11.0-RA1-T-nocloud','WRF_COL_CRM']

m = len(models)
cols =  np.load('color_grid.npy', allow_pickle = True)

pl = 0.22 # pos limit
nlim= -0.07 # neg limit
bs = 0.001 # bin size

temps = ['large295', 'large300', 'large305']

bins = np.arange(nlim, pl, bs) #creates bins
b = len(bins)
print('no of vert vel bins: ', b)
mnbin = [] # create array of the mid-point between bin limits for plotting
for i in range(1,b):
    mnbin = np.append(mnbin, np.mean([bins[i-1], bins[i]]))

no of vert vel bins:  291


In [4]:
#choose model level - all analysis done at 500hPa

pLev = 500
deltaT = 5

In [5]:
directLW, decomposedLW = np.empty((2, m)),np.empty((2, m))
directSW, decomposedSW = np.empty((2, m)),np.empty((2, m))
k=0
for model in models:
    print(model)
    dp = '/home/users/arm33/RCEMIP/'+ model +'/processed_new/3D/'
    dp3 = '/home/users/arm33/RCEMIP/'+ model +'/processed_new/2D/large'

    for i in range(3): # loop over temperatures to get the wa fields
        pa = np.load(dp + temps[i] + 'pa_profile_25d.npy', allow_pickle = True)/100
        if i==0:
            wa500_alltemp = np.empty((3,np.shape(pa)[0],np.shape(pa)[2], np.shape(pa)[3]))
        wa = np.load(dp + temps[i] + 'wa_profile_25d.npy', allow_pickle = True)
        wa500 = funcs.getPlevel(wa, pa, pLev)             

        #stack up wa500 for use in dynamic effect
        wa500_alltemp[i,] = wa500  

        if i==1 or i==2:
            if i==1:
                con,new = '295', '300'
                marker = 'o'

            if i ==2:
                con,new = '300', '305'
                marker = 's'


            #calc dynamic effect with 3D data
            lwcrf1 = np.load(dp3 + con + 'lwcrf24hrs.npy', allow_pickle = True)
            lwcrf2 = np.load(dp3 + new + 'lwcrf24hrs.npy', allow_pickle = True)

            swcrf1 =  np.load(dp3 + con + 'swcrf24hrs.npy', allow_pickle = True)
            swcrf2 =  np.load(dp3  + new + 'swcrf24hrs.npy', allow_pickle = True)

            net1 = lwcrf1 + swcrf1
            net2 = lwcrf2 + swcrf2 

            wa5001 = wa500_alltemp[i-1,]
            wa5002 = wa500_alltemp[i,]
            if np.max(wa5001) > pl or np.max(wa5002) > pl:
                print(model + ' exceeds max')
            if np.min(wa5001) < nlim or np.min(wa5002) < nlim:
                print(model + 'exceeds min')
                
            #control functions
            dig1 = np.digitize(wa5001, bins)
            A1 = funcs.createA(dig1, b)
            LW1 = funcs.createR(lwcrf1, dig1, b)
            LW1 = funcs.rpTrailingZeros(A1, LW1)
            SW1 = funcs.createR(swcrf1, dig1, b)
            SW1 = funcs.rpTrailingZeros(A1, SW1)
            NET1 = funcs.createR(net1, dig1, b)
            NET1 = funcs.rpTrailingZeros(A1, NET1) 

            #new functions
            dig2 = np.digitize(wa5002, bins)
            A2 = funcs.createA(dig2, b)
            LW2 = funcs.createR(lwcrf2, dig2, b)
            LW2 = funcs.rpTrailingZeros(A2, LW2)
            SW2 = funcs.createR(swcrf2, dig2, b)
            SW2 = funcs.rpTrailingZeros(A2, SW2)
            NET2 = funcs.createR(net2, dig2, b)
            NET2 = funcs.rpTrailingZeros(A2, NET2)

            dLW = LW2 - LW1
            dSW = SW2 - SW1
            dNET = NET2 - NET1
            dA = A2 - A1

            totLW,th,LWdyn,nl = funcs.decompose(A1, A2, LW1, LW2, deltaT)
            totSW,th,SWdyn,nl = funcs.decompose(A1, A2, SW1, SW2, deltaT)
            #tot,th,NETdyn,nl = funcs.decompose(A1, A2, NET1, NET2, deltaT) 
            print(LWdyn, SWdyn)

            # direct calcuation of cloud feedback
            directLW[i-1,k] = (np.mean(lwcrf2) - np.mean(lwcrf1))/deltaT
            decomposedLW[i-1,k] = totLW
            directSW[i-1,k] = (np.mean(swcrf2) - np.mean(swcrf1))/deltaT
            decomposedSW[i-1,k] = totSW
                
    k+=1


CM1


/home/users/arm33/RCEMIP/allMod/../funcs.py:120: RuntimeWarning: Mean of empty slice
  arrbin = [np.nanmean(arr[dig==i]) for i in range(1,b)]


-0.14696530286305473 0.1310530835137402
0.004213639554707177 0.04217776165144234
dam
0.02630774557845611 -0.02003295124841226
0.03575136963591652 -0.03679196976052028
ICON_LEM_CRM
0.16086336115490468 -0.14322901820127304
0.1611393077195911 -0.12817565951308524
ICON_NWP_CRM
0.3869077127751786 -0.2780306515681995
0.03649288217873302 -0.009858593657743641
MESONH
-0.011838760568374008 -0.010279033629238654
MESONHexceeds min
1.148882426059264 -1.7421088251948755
SAM_CRM
-0.15638102203940218 0.25469692638830277
0.023138291374404967 -0.02224433293335819
SCALE
-0.01244253800972379 0.014160019144775049
-0.013380345359408125 0.014745176565205098
UCLA-CRM
0.04851253068431458 -0.2691991820417502
0.08292892744373687 -0.23515739628020071
UKMOi-vn11.0-CASIM
-0.06761244905996179 0.1537075230880645
0.022602873804934337 -0.02705799788418196
UKMOi-vn11.0-RA1-T
-0.29691360165704717 0.3403492549589755
-0.2482662328489016 0.3338399726791786
UKMOi-vn11.0-RA1-T-nocloud
-0.22529999133692816 0.3477695011971127


In [30]:
difference = decomposedLW - directLW
print('LW mean difference: ', np.mean(difference))
print('LW range of differences: ', np.min(difference), np.max(difference))
print('RMSE: ', funcs.calcRMSE(decomposedLW, directLW))

LW mean difference:  -0.010028818324425747
LW range of differences:  -0.03903419260824936 6.661338147750939e-16
RMSE:  0.015476553117721698


In [31]:
difference = decomposedSW - directSW
print('SW mean difference: ', np.mean(difference))
print('SW range of differences: ', np.min(difference), np.max(difference))
print('RMSE: ', funcs.calcRMSE(decomposedSW, directSW))

SW mean difference:  0.01449338685045761
SW range of differences:  -3.469446951953614e-16 0.05774823170323562
RMSE:  0.022306103034159794
